# pc_to_tex
## Usage


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import struct
#from google.colab import drive

In [12]:
#file_path = "drive/My Drive/Colab Notebooks/pc_to_tex/"
file_path = "./"
pc = pd.read_table(file_path + "akihabara.asc", sep=" ", header=None, names=("y", "z", "x", "r", "g", "b"))
tex_width = 1024

In [13]:
def save_tex(r, c, tex_width, tex_num):
    pos_tex = np.pad(c, ((0, tex_width * 2 - c.shape[0]), (0,0), (0,0)), "constant")
    pos_tex = Image.fromarray(np.uint8(np.round(pos_tex)))
    pos_tex.save(file_path + "pos" + str(tex_num) + ".png")

In [14]:
tex_num = 0

for i, pos in enumerate(zip(pc["x"], pc["y"], pc["z"])):
    if i % tex_width ** 2 == 0:
        if not i == 0:
            save_tex(r, c, tex_width, tex_num)
            tex_num += 1
        
        r = np.empty((2, 0, 3))
        c = np.empty((0, tex_width * 2, 3))
    
    if i % tex_width * 2 == 0:
        if not i % tex_width ** 2== 0:
            c = np.append(c, r, axis=0)
            r = np.empty((2, 0, 3))
    
    a = np.empty((2, 2, 0))
    
    for xyz in pos:
        xs = struct.pack('>f', xyz)
        xn = struct.unpack('>L', xs)[0]
        b = np.array([[[xn >> 0 & 0xff],[xn >> 8 & 0xff]],
                      [[xn >> 16 & 0xff],[xn >> 24 & 0xff]]])
        a = np.append(a, b, axis=2)
    
    r = np.append(r, a, axis=1)

if r.shape[1] > 0:
    r = np.pad(r, ((0,0),(0,tex_width * 2 - r.shape[1]),(0,0)), "constant")
    c = np.append(c, r, axis=0)

save_tex(r, c, tex_width, tex_num)

In [15]:
col = pc[["r", "g", "b"]].values.reshape([-1, 1, 3])
tex_num = np.ceil((len(col)) / tex_width ** 2)
tex_length = int(tex_num * (tex_width ** 2) - len(col))
col_tex = np.pad(col, ((0,tex_length),(0,0),(0,0)), "constant")
col_tex = np.array_split(col_tex, tex_num)

for i, tex in enumerate(col_texs):
    col_tex = np.reshape(tex, (tex_width, tex_width, 3))
    col_tex = Image.fromarray(np.uint8(np.round(col_tex)))
    col_tex.save(file_path + "col" + str(i) + ".png")